Using the topic models, I create a ScatterText graph for every topic to contrast how Republicans and Democrats talk about each topic differently

In [ ]:
from pymongo import MongoClient
config = {
    'host': 'xx.xxxx.xxxx.xx',
    'username': 'xxxxxxxx',
    'password': 'xxxxxxxx',
    'authSource': 'cool'
}

client = MongoClient(**config)
db = client.cool

In [ ]:
db.bills.count_documents({})

In [ ]:
import pandas as pd
import numpy as np
import re

Again, I use stop words to preprocess the text of each bill and prepare them for making the ScatterText

In [ ]:
real_stops = ['secretary', 'insert','inserting','striking','program','plan','note','rule','agency',
             'administration','subparagraph','later','services','appropriate','administrator',
             'commission','faa', 'national','usc','assistance','eligible','service','clause',
             'development','respect','apply','case','percent','determined','board','subtitle',
             'person','period','office','use','programs','individuals','systems','revies','study',
             'public','management','administering','imposed','authority','relating','state',
             'federal','project','law','review','central','agencies','subsection','san',
             'subsection','activities','attorney','department','resolution','joint','rules',
             'date','effect','described','motion','report','major','days','day','consideration',
             'order', 'amended','paragraph','entities','counsel','mr','ms','proposed','information',
             'local','written','affairs', 'non','year','month','members','committees','government',
             'head','member','general','shall','action','assessment','subject','including',
             'available','purposes','term','provided','remain','necessary','used','end','president',
             'zzz','zz','zuni','zuhair','zte','zour','zou','zoster','zoris','zor', 'heading','subheading',
             'chapter','change','new','subchapter','numerical','sequence','designed','appropriated',
             'september',]

In [ ]:
from sklearn.feature_extraction import text
stops = text.ENGLISH_STOP_WORDS
stops = list(stops) + real_stops

In [ ]:
def text_clean(text, stops):
    text = re.sub('\\n', ' ', text)
    text = re.sub('_', ' ', text)
    text = re.sub('\[.{,50}\]', ' ', text)
    text = re.sub('\<.{,10}\>', ' ', text)
    text = re.sub('[.\'`,;():\-$%&\^#?!><]', ' ', text).lower()
    text = re.sub('\w*\d+\w*', '', text)
    text = re.sub('\s[xvil]+\s', ' ', text)
    text = re.sub('\s{2,}', ' ', text)
    for stop in stops:
        text = re.sub('\s{}\s'.format(stop), ' ', text)
    return text

I read in the csv I made after topic modeling and filter for each specific topic in order to make the ScatterText

In [ ]:
df = pd.read_csv('bill_details.csv', index_col=[0])

In [ ]:
df.topic.unique()

I run the following code for all 15 topics and merely change the mask to filter for each topic. I end up with 15 ScatterTexts which I refer to for the final flask app.

In [ ]:
tax_index = df[df.topic=='education'].index

In [ ]:
cursor = db.bills.find({})
trump = []
for bill in list(cursor):
    if bill['_id'] in tax_index:
        bill['bill'] = text_clean(bill['bill'], stops)
        if len(bill['bill']) > 6000:
            bill['bill'] = bill['bill'][:6000]
            trump.append(bill)
        else:
            trump.append(bill)

In [ ]:
len(trump)

In [ ]:
trump_df = pd.DataFrame(trump)

In [ ]:
trump_df[['party','bill']].head()

In [ ]:
import scattertext as st

In [ ]:
corpus = st.CorpusFromPandas(trump_df[['party', 'bill']],
                             category_col='party',
                             text_col='bill',
                             nlp=st.whitespace_nlp_with_sentences,
                             verbose=True
                            ).build()

In [ ]:
html = st.produce_scattertext_explorer(
        corpus,
        category='D',
        category_name='Democratic',
        not_category_name='Republican',
        minimum_term_frequency=100,
        pmi_threshold_coefficient=5,
        width_in_pixels=1000,
        )

In [ ]:
open('education.html', 'wb').write(html.encode('utf-8'));